<a href="https://colab.research.google.com/github/axel-sirota/practical-nlp/blob/main/1-synonims/Practical_NLP_2_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Oct 13 17:42:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import multiprocessing
import tensorflow as tf
import sys
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from textblob import TextBlob, Word
from keras_preprocessing.sequence import pad_sequences
from keras.initializers import Constant
import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk

TRACE = False
embedding_dim = 100
epochs=2
batch_size = 500
BATCH = True

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config) 
  K.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')
nltk.download('punkt')
tokenizer = lambda x: TextBlob(x).words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
%%writefile get_data.sh
if [ ! -f yelp.csv ]; then
  wget -O yelp.csv https://www.dropbox.com/s/xds4lua69b7okw8/yelp.csv?dl=0
fi

if [ ! -f glove.6B.100d.txt ]; then
  wget -O glove.6B.100d.txt https://www.dropbox.com/s/dl1vswq2sz5f1ws/glove.6B.100d.txt?dl=0
fi

Writing get_data.sh


In [5]:
!bash get_data.sh

--2022-10-13 17:42:59--  https://www.dropbox.com/s/xds4lua69b7okw8/yelp.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/xds4lua69b7okw8/yelp.csv [following]
--2022-10-13 17:42:59--  https://www.dropbox.com/s/raw/xds4lua69b7okw8/yelp.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc165ad61e2069d6eccc096f14f1.dl.dropboxusercontent.com/cd/0/inline/BuxyjeiJrE64HslhF316G3i0TpfpaDj9tTUY5TbUzB5LO4lFqqTFpdWGNhRfBSbkZqiLfShIJIa73vZK7H0_L9AHyIio8IQZBuJgNUJSoPHJBUz1SMU-lh0Pz_3qeR-chgeha0s5fhiKGRplzpQHG68vwwm9B55-RvWqmDWiGE6bCA/file# [following]
--2022-10-13 17:43:00--  https://uc165ad61e2069d6eccc096f14f1.dl.dropboxusercontent.com/cd/0/inline/BuxyjeiJrE64HslhF316G3i0TpfpaDj9tTUY5TbUzB5LO4lFqqTFpdWGNhRfBSbkZqiLfShIJIa73vZK7H0_L9A

In [6]:
path = './yelp.csv'
yelp = pd.read_csv(path)
# Create a new DataFrame that only contains the 5-star and 1-star reviews.
yelp_best_worst = yelp[(yelp.stars==5) | (yelp.stars==1)]
X = yelp_best_worst.text
y = yelp_best_worst.stars.map({1:0, 5:1})

In [7]:
corpus = [sentence for sentence in X.values if type(sentence) == str and len(TextBlob(sentence).words) > 3]

In [8]:
path_to_glove_file = "./glove.6B.100d.txt"
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400001 word vectors.


In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
tokenized_corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
vocab_size = len(tokenizer.word_index) + 1

In [10]:
print(f'First 5 corpus items are {corpus[:5]}')
print(f'Length of corpus is {len(corpus)}')

First 5 corpus items are ['My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I\'ve ever had.  I\'m pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I\'ve ever had.\n\nAnyway, I can\'t wait to go back!', 'I have no idea why some people give bad reviews about this place. It g

In [11]:
num_tokens = vocab_size + 1
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 17190 words (2759 misses)


In [12]:
model = Sequential()
model.add(Embedding(input_dim=num_tokens, output_dim=embedding_dim, input_length=4, embeddings_initializer=Constant(embedding_matrix),
    trainable=False))
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embedding_dim,)))
model.add(Dense(vocab_size, activation='softmax'))

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            1995100   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 19950)             2014950   
                                                                 
Total params: 4,010,050
Trainable params: 2,014,950
Non-trainable params: 1,995,100
_________________________________________________________________


In [14]:
def generate_data(corpus, vocab_size, window_size=2, sentence_batch_size=10,  batch_size=250):
    np.random.shuffle(np.array(corpus))
    number_of_sentence_batches = (len(corpus) // sentence_batch_size) + 1
    for batch in range(number_of_sentence_batches):
        lower_end = batch*batch_size
        upper_end = (batch+1)*batch_size if batch+1 < number_of_sentence_batches else len(corpus)
        mini_batch_size = upper_end - lower_end
        maxlen = window_size*2
        X = []
        Y = []
        for review_id, words in enumerate(corpus[lower_end:upper_end]):
            L = len(words)
            for index, word in enumerate(words):
                contexts = []
                labels   = []            
                s = index - window_size
                e = index + window_size + 1

                contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
                labels.append(word)

                x = pad_sequences(contexts, maxlen=maxlen)
                y = np_utils.to_categorical(labels, vocab_size)
                X.append(x)
                Y.append(y)
        X = tf.constant(X)
        Y = tf.constant(Y)
        number_of_batches = len(X) // batch_size
        for real_batch in range(number_of_batches):
          lower_end = batch*batch_size
          upper_end = (batch+1)*batch_size
          batch_X = tf.squeeze(X[lower_end:upper_end])
          batch_Y = tf.squeeze(Y[lower_end:upper_end])
          yield (batch_X, batch_Y)

In [15]:
def fit_model():
    if not BATCH:
        X, Y = generate_data(corpus=tokenized_corpus, vocab_size=vocab_size, batch_size=len(X))
        print(f'Size of X is {X.shape} and Y is {Y.shape}')
        model.fit(X, Y, epochs = epochs)
    else:
        for epoch in range(epochs):
          index = 1
          for x, y in generate_data(corpus=tokenized_corpus, vocab_size=vocab_size, batch_size=batch_size):
              print(f'Training on Epoch/Iteration: {epoch+1}/{index}')
              index += 1
              history = model.train_on_batch(x, y, reset_metrics=False, return_dict=True)
              print(history)

In [ ]:
fit_model()

Training on Epoch/Iteration: 1/1
{'loss': 9.899017333984375}
Training on Epoch/Iteration: 1/2
{'loss': 9.874351501464844}
Training on Epoch/Iteration: 1/3
{'loss': 9.849365234375}
Training on Epoch/Iteration: 1/4
{'loss': 9.82418441772461}
Training on Epoch/Iteration: 1/5
{'loss': 9.798870086669922}
Training on Epoch/Iteration: 1/6
{'loss': 9.773459434509277}
Training on Epoch/Iteration: 1/7
{'loss': 9.74797534942627}
Training on Epoch/Iteration: 1/8
{'loss': 9.722434043884277}
Training on Epoch/Iteration: 1/9
{'loss': 9.696846008300781}
Training on Epoch/Iteration: 1/10
{'loss': 9.671223640441895}
Training on Epoch/Iteration: 1/11
{'loss': 9.645570755004883}
Training on Epoch/Iteration: 1/12
{'loss': 9.619895935058594}
Training on Epoch/Iteration: 1/13
{'loss': 9.594202995300293}
Training on Epoch/Iteration: 1/14
{'loss': 9.568495750427246}
Training on Epoch/Iteration: 1/15
{'loss': 9.542778968811035}
Training on Epoch/Iteration: 1/16
{'loss': 9.51705551147461}
Training on Epoch/Itera